In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
from area import area
from pyproj import Proj
from shapely.geometry import shape
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext







In [2]:

df = pd.read_csv("zameen-property-data - zameen-property-data.csv")
df.head()

/home/amiable/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms,date_added,area-cleaned,square-feet,area.1
0,237062,3325,https://www.zameen.com/Property/g_10_g_10_2_gr...,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,4 Marla,For Sale,2,02-04-2019,4,900.0,4
1,346905,3236,https://www.zameen.com/Property/e_11_2_service...,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,5.6 Marla,For Sale,3,05-04-2019,5.6,1260.0,5.6
2,386513,764,https://www.zameen.com/Property/islamabad_g_15...,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,8 Marla,For Sale,5,07-17-2019,8,1800.0,8
3,656161,340,https://www.zameen.com/Property/islamabad_bani...,House,43500000,Bani Gala,Islamabad,Islamabad Capital,33.707573,73.151199,4,2 Kanal,For Sale,4,04-05-2019,2,450.0,2 Kanal
4,841645,3226,https://www.zameen.com/Property/dha_valley_dha...,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,8 Marla,For Sale,3,07-10-2019,8,1800.0,8


In [3]:
df = df[~df['area.1'].str.contains('Kanal')]

In [4]:
df.head()

,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms,date_added,area-cleaned,square-feet,area.1
0,237062,3325,https://www.zameen.com/Property/g_10_g_10_2_gr...,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,4 Marla,For Sale,2,02-04-2019,4,900.0,4
1,346905,3236,https://www.zameen.com/Property/e_11_2_service...,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,5.6 Marla,For Sale,3,05-04-2019,5.6,1260.0,5.6
2,386513,764,https://www.zameen.com/Property/islamabad_g_15...,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,8 Marla,For Sale,5,07-17-2019,8,1800.0,8
4,841645,3226,https://www.zameen.com/Property/dha_valley_dha...,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,8 Marla,For Sale,3,07-10-2019,8,1800.0,8
7,1258636,3241,https://www.zameen.com/Property/e_11_e_11_4_ap...,Flat,7800000,E-11,Islamabad,Islamabad Capital,33.698244,72.984238,2,6.2 Marla,For Sale,2,05-05-2019,6.2,1395.0,6.2


In [5]:
df1 = df.drop(['page_url','area','area.1','latitude','longitude','date_added','province_name','location_id','purpose'],axis='columns')
df1.head()

,property_id,property_type,price,location,city,baths,bedrooms,area-cleaned,square-feet
0,237062,Flat,10000000,G-10,Islamabad,2,2,4,900.0
1,346905,Flat,6900000,E-11,Islamabad,3,3,5.6,1260.0
2,386513,House,16500000,G-15,Islamabad,6,5,8,1800.0
4,841645,House,7000000,DHA Defence,Islamabad,3,3,8,1800.0
7,1258636,Flat,7800000,E-11,Islamabad,2,2,6.2,1395.0


In [6]:
df2 = df1.dropna()
df2.head()

,property_id,property_type,price,location,city,baths,bedrooms,area-cleaned,square-feet
0,237062,Flat,10000000,G-10,Islamabad,2,2,4,900.0
1,346905,Flat,6900000,E-11,Islamabad,3,3,5.6,1260.0
2,386513,House,16500000,G-15,Islamabad,6,5,8,1800.0
4,841645,House,7000000,DHA Defence,Islamabad,3,3,8,1800.0
7,1258636,Flat,7800000,E-11,Islamabad,2,2,6.2,1395.0


In [7]:
df2.bedrooms

0         2
1         3
2         5
4         3
7         2
         ..
168441    6
168442    3
168443    6
168444    3
168445    3
Name: bedrooms, Length: 137769, dtype: int64

In [8]:
df3 = df2.drop(['property_type','city'],axis="columns")

In [9]:
df3 = df3[['price', 'location','baths','bedrooms','area-cleaned','square-feet']]


In [10]:
location = pd.get_dummies(df3.location)
location.head()

,12th Avenue,204 Chak Road,7th Avenue,9th Avenue,APP Employees Co-operative Housing Society,AR Cottages,ASF Housing Scheme,ASF Tower,AWT Army Welfare Trust,Aabpara Coop Housing Society,...,Zamzama,Zaraj Housing Scheme,Zeeshan Colony,Zero Point,Zia Colony,Zone 5,Zubaida Park,Zubair Colony,Zulfiqar Colony,circular road
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df4 = pd.concat([df3,location],axis="columns")
df4



,price,location,baths,bedrooms,area-cleaned,square-feet,12th Avenue,204 Chak Road,7th Avenue,9th Avenue,...,Zamzama,Zaraj Housing Scheme,Zeeshan Colony,Zero Point,Zia Colony,Zone 5,Zubaida Park,Zubair Colony,Zulfiqar Colony,circular road
0,10000000,G-10,2,2,4,900.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6900000,E-11,3,3,5.6,1260.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16500000,G-15,6,5,8,1800.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7000000,DHA Defence,3,3,8,1800.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7800000,E-11,2,2,6.2,1395.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168441,26500000,Gadap Town,0,6,9.6,2160.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168442,12500000,Gadap Town,0,3,8,1800.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168443,27000000,Gadap Town,0,6,9.6,2160.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168444,11000000,Gadap Town,0,3,7.8,1755.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df5 = df4.drop('location',axis='columns')
df5['square-feet'] = df5['square-feet'].astype(int)


In [13]:
df6 = df5[~(df5['square-feet']/df5['bedrooms']<300)]


In [14]:
X = df6.drop('price',axis='columns')
X.head()

,baths,bedrooms,area-cleaned,square-feet,12th Avenue,204 Chak Road,7th Avenue,9th Avenue,APP Employees Co-operative Housing Society,AR Cottages,...,Zamzama,Zaraj Housing Scheme,Zeeshan Colony,Zero Point,Zia Colony,Zone 5,Zubaida Park,Zubair Colony,Zulfiqar Colony,circular road
0,2,2,4,900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,3,5.6,1260,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,5,8,1800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,3,8,1800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2,2,6.2,1395,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y = df6.price

print(X.shape)
print(y.shape)


(115963, 1470)
(115963,)


In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.29,random_state=10)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


# lr = Lasso()
lr = Lasso()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

import time 
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))


In [18]:
import pickle
with open('zamee_prices_model.pickle','wb') as f:
    pickle.dump(lr,f)

In [19]:
def predict_price(location,square_feet,baths,bedrooms):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = square_feet
    x[1] = baths
    x[2] = bedrooms
    if loc_index >= 0:
        x[loc_index] = 1

    return lr.predict([x])[0]

In [20]:
predict_price("Cantt",1000,2,2)

1158852838.4010682

In [21]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
pipe = Pipeline([('imputer',SimpleImputer()),('model',Ridge())])

In [23]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('model', Ridge())])

In [24]:
pipe.predict(X_test)

array([11647939.78318171,  1935554.77812571, 17617171.97883364, ...,
        1443202.67739033,  6274899.23783712, 11019162.82804692])

In [27]:
from sklearn2pmml import sklearn2pmml,make_pmml_pipeline
sklearn2pmml(make_pmml_pipeline(pipe),"pipe_pmml.pmml")

/home/amiable/.local/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
/usr/lib/python3.8/subprocess.py:844: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
/usr/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)
